# Post-process mizuRoute daily site selected history files


In [1]:
%matplotlib inline

import os, sys
import xarray as xr
import pandas as pd
import numpy as np

print("\nThe Python version: %s.%s.%s" % sys.version_info[:3])
print(xr.__name__, xr.__version__)


The Python version: 3.10.12
xarray 2023.6.0


## 1. Directories

In [2]:
main_path = './'
ancillary_path = os.path.join(main_path, 'ancillary_data')
output_path = os.path.join(main_path, 'data', 'summa_mizuRoute')
mizuRoute_path  = '/glade/u/home/mizukami/proj/pnw-extrems/models/PNW_GCM'

## 2. Setups

In [7]:
sim_cases = {
        #       'gmet_adjusted_t_fixed_4subregion_3_run43':  {'renaming':'gmet', }, 
               'icar_CanESM5_hist':       {'renaming':'icar_CanESM5_hist',}, 
               'icar_CanESM5_ssp370':     {'renaming':'icar_CanESM5_ssp370',},
               'icar_NorESM2-MM_ssp370':  {'renaming':'icar_NorESM2-MM_ssp370',},
               'icar_NorESM2-MM_ssp585':  {'renaming':'icar_NorESM2-MM_ssp585',},
               'icar_NorESM2-MM_hist':    {'renaming':'icar_NorESM2-MM_hist',},
               'icar_MPI-M.MPI-ESM1-2-LR_hist':    {'renaming':'icar_MPI-ESM1-2-LR_hist',},
               'icar_MPI-M.MPI-ESM1-2-LR_ssp370':    {'renaming':'icar_MPI-ESM1-2-LR_ssp370',},
            }

sites = ['LIB']

english_unit=True

In [8]:
cms2cfs=35.3147

## 3. Read ancillary data, e.g.,  flow sites geopackages, ascii

### Flow site name to be plotted

In [9]:
site_name = pd.read_csv(os.path.join(ancillary_path, 'meta_data.pnnl_sites.v3.txt'), delim_whitespace=True)
if bool(sites):
    site_name = site_name[site_name['label'].isin(sites)] #
site_name

,reachID,hruId,label
28,78013865,170101011208,LIB


----
## 4. Read simulated flow time series

In [10]:
%%time
print('---- Read validation period simulated flow ----')

site_array = np.full(len(site_name), 'nan', dtype=object)

for case, meta in sim_cases.items():
    nclist = os.path.join(mizuRoute_path, 'cases', f"run_{case}/archived/run_{case}.h.*_selected.nc")
    ds_tmp = xr.open_mfdataset(nclist, data_vars='minimal').compute()
    
    #select reach matching site list
    ds_tmp = ds_tmp.where(ds_tmp['reachID'].isin(list(site_name['reachID'])), drop=True)

    # compute daily mean
    ds_tmp = ds_tmp.resample(time='1D').mean()
    ds_tmp['reachID'] = ds_tmp['reachID'].isel(time=0, drop=True)
    
    # get site name
    for ix, reach_id in enumerate(ds_tmp['reachID'].values):
        site = site_name.loc[site_name['reachID']==reach_id, 'label'].values[0]
        site_array[ix] = site
    ds_tmp['site'] = xr.DataArray(site_array, dims=('seg'))
    
    # house keeping...
    ds_tmp = ds_tmp.rename({'IRFroutedRunoff':'streamflow'})
    ds_tmp['reachID'] = ds_tmp['reachID'].astype('int32')
    
    if english_unit:
        ds_tmp['streamflow'] = ds_tmp['streamflow']*cms2cfs
        ds_tmp['streamflow'].attrs['units'] = 'cfs'

    ds_tmp.to_netcdf(os.path.join(output_path, f"{meta['renaming']}.nc"))
    print('%s'%case)

---- Read validation period simulated flow ----
icar_MPI-M.MPI-ESM1-2-LR_ssp370
CPU times: user 257 ms, sys: 367 ms, total: 624 ms
Wall time: 3.16 s
